In [1]:
from pyspark.sql import SparkSession
from  pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [ ]:
spark =(
    SparkSession
        .builder
        .master('local[*]')
        .appName('bucketing')
        .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")
spark.conf.set('spark.sql.autoBroadcastJoinThreshold',-1)

25/06/22 11:15:38 WARN Utils: Your hostname, hari-HP-Laptop-15s-du2xxx resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
25/06/22 11:15:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/06/22 11:15:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [67]:
spark

In [98]:
titanic_json =(
    spark.read.json('./data/titanic-parquet.json')
)

In [39]:
titanic_json.printSchema()
titanic_json.show(20,False)

root
 |-- Age: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- PassengerId: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Ticket: string (nullable = true)

+----+-----+--------+-------+-------------------------------------------------------+-----+-----------+------+------+-----+--------+----------------+
|Age |Cabin|Embarked|Fare   |Name                                                   |Parch|PassengerId|Pclass|Sex   |SibSp|Survived|Ticket          |
+----+-----+--------+-------+-------------------------------------------------------+-----+-----------+------+------+-----+--------+----------------+
|22.0|null |S       |7.25   |Braund, Mr. Owen Harris                                |0    |1

In [41]:
titanic_json.select('Age').distinct().show()

+----+
| Age|
+----+
| 8.0|
|70.0|
| 7.0|
|20.5|
|49.0|
|29.0|
|40.5|
|64.0|
|47.0|
|42.0|
|24.5|
|44.0|
|35.0|
|null|
|62.0|
|18.0|
|80.0|
|34.5|
|39.0|
| 1.0|
+----+
only showing top 20 rows



In [75]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold',-1)

In [80]:
titanic_json.rdd.getNumPartitions()

1

In [79]:
mean_age = titanic_json.select(F.round(F.mean(F.col('Age')),2).alias('mean_age'))

mean_age.explain(True)


== Parsed Logical Plan ==
'Project [round(avg('Age), 2) AS mean_age#806]
+- Relation[Age#7,Cabin#8,Embarked#9,Fare#10,Name#11,Parch#12,PassengerId#13,Pclass#14,Sex#15,SibSp#16,Survived#17,Ticket#18] json

== Analyzed Logical Plan ==
mean_age: double
Aggregate [round(avg(Age#7), 2) AS mean_age#806]
+- Relation[Age#7,Cabin#8,Embarked#9,Fare#10,Name#11,Parch#12,PassengerId#13,Pclass#14,Sex#15,SibSp#16,Survived#17,Ticket#18] json

== Optimized Logical Plan ==
Aggregate [round(avg(Age#7), 2) AS mean_age#806]
+- Project [Age#7]
   +- Relation[Age#7,Cabin#8,Embarked#9,Fare#10,Name#11,Parch#12,PassengerId#13,Pclass#14,Sex#15,SibSp#16,Survived#17,Ticket#18] json

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[avg(Age#7)], output=[mean_age#806])
+- Exchange SinglePartition, true, [id=#421]
   +- *(1) HashAggregate(keys=[], functions=[partial_avg(Age#7)], output=[sum#810, count#811L])
      +- FileScan json [Age#7] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileI

In [106]:
print((   
    titanic_json.crossJoin(mean_age)
    .withColumn('Age',F.coalesce(F.col('Age'),F.col('mean_age')))
    .drop(F.col('mean_age'))
).rdd.toDebugString())

b'(1) MapPartitionsRDD[359] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  MapPartitionsRDD[358] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  SQLExecutionRDD[357] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  MapPartitionsRDD[356] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  MapPartitionsRDD[355] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  UnsafeCartesianRDD[354] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  MapPartitionsRDD[347] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  FileScanRDD[346] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  MapPartitionsRDD[353] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  ShuffledRowRDD[352] at javaToPython at NativeMethodAccessorImpl.java:0 []\n +-(1) MapPartitionsRDD[351] at javaToPython at NativeMethodAccessorImpl.java:0 []\n    |  MapPartitionsRDD[350] at javaToPython at NativeMethodAccessorImpl.java:0 []\n    |  MapPartitionsRDD[34

+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----------+---------------+-------+----+----+----+----+----+----+----+-------+----+----+-------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+-----------+----+----+----+----+----+----+----+----+----+-------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+
| Age|null| A10| A14| A16| A19| A20| A23| A24| A26| A31| A32| A34| A36|  A5|  A6|  A7|B101|B102| B18| B19| B20| B22| B28|  B3| B30| B35| B37| B38| B39|  B4| B41| B42| B49|  B5| B50|B51 B53 B55|B57 B59 B63 B6